In [1]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/notebooks/beam/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s2'

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from RIMEz import beam_models
import numpy as np
import healpy as hp
from spin1_beam_model import cst_processing, jones_matrix_field
import rotations
import parser

In [3]:
processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
)

In [4]:
processor.compute_spin1_harmonics()
processor.write_model_data(file_path, beam_destination)
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [12]:
nside = 128
source = parser.obj_catalog[999]
#theta, phi = hp.pix2ang(nside,np.arange(12 * nside * nside))
ra = np.radians(source.ra_angle)
dec = np.radians(source.dec_angle)
az, alt = rotations.eq_to_topo(ra, dec,
    np.radians(rotations.hera_lat),
    rotations.get_lst(), radians=True)

# reformat into arrays so that the spline function can handle the single angles
az = np.array([az])
alt = np.array([alt])
az, alt

(array([-2.89801852]), array([0.78018452]))

In [13]:
#az = phi
#alt = np.pi / 2 - theta
J = spline_beam_func(150e6, alt, az)
J.shape

(1, 2, 2)

In [17]:
# this is a module purely for debugging purposes
# Jupyter notebook fails to register bug fixes in imported code,
# even if you run the import line again. So, we use this:
import importlib
importlib.reload(parser)
# To properly reload the import without having to restart

<module 'parser' from '/Users/lfinkbeiner/Documents/GitHub/HERA/notebooks/visibility/parser.py'>

In [18]:
print(parser.obj_catalog[999])
print(parser.visibility_integrand(J, parser.obj_catalog[999], 151e6))

Name: GLEAM J070608-734534
Right ascension: 106.53525
Declination: -72.24030555555555
151 MHz flux: 116.260



TypeError: Cannot cast array data from dtype('complex128') to dtype('<U64') according to the rule 'safe'